In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.layers import BatchNormalization

import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [3]:
#train_df.columns = train_df.columns.str.strip()
#test_df.columns = test_df.columns.str.strip()
#val_df.columns = val_df.columns.str.strip()

In [4]:
image_directory = 'baggage/train/'


#Now let us read metadata to get our Y values (multiple lables)
df = pd.read_csv('baggage/train_file/_classes.csv')    
print(df.head())     # printing first five rows of the file
print(df.columns)

                                            filename   Gun   Knife   Pliers  \
0  P00392_jpg.rf.927a720cb4be08db58c89aa0bafe58ba...     1       1        0   
1  P08714_jpg.rf.92ae9225b26d09c6efba68704c723e65...     0       0        1   
2  P06862_jpg.rf.92b47b9cee6b711d9d26245b83cf3a27...     0       0        0   
3  P00532_jpg.rf.92bc280cf669e4fd66d9f3b26a036c3c...     0       1        0   
4  P08428_jpg.rf.92c57b5d947cda38d974ba0266aaf66d...     0       0        1   

    Scissors   Wrench  
0          0        0  
1          0        0  
2          1        0  
3          0        0  
4          0        0  
Index(['filename', ' Gun', ' Knife', ' Pliers', ' Scissors', ' Wrench'], dtype='object')


In [5]:
df = df.iloc[:100]  #Loading only first 1000 datapoints for memory reasons 
#Need to read images using the tag from metadata.
#Otherwise, if read directly from the folder then images may not correspond to 
#the metadata from the csv file.

In [6]:
y = np.array(df)

In [13]:
SIZE = 640
X_dataset = []  
for i in tqdm(range(df.shape[0])):
    img = image.load_img(image_directory +df['filename'][i], target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
    
X = np.array(X_dataset)

100%|██████████| 100/100 [00:02<00:00, 49.81it/s]


In [16]:
X.shape

(100, 640, 640, 3)

In [ ]:
#import os
#import tensorflow as tf
#from tensorflow.keras.preprocessing import image
#from tqdm import tqdm
#
#SIZE = 640
#image_directory = 'baggage/train'
#df_length = df.shape[0]
#
#X_dataset = []
#
#for i in tqdm(range(df_length)):
#    filename = df.iloc[i]['filename']
#    img_path = os.path.join(image_directory, filename)
#    img = image.load_img(img_path, target_size=(SIZE, SIZE, 3))
#    img = image.img_to_array(img)
#    img = img / 255.0
#    X_dataset.append(img)
#
#X_dataset = tf.stack(X_dataset)
#

100%|██████████| 1000/1000 [00:17<00:00, 57.31it/s]

In [17]:
X = np.array(X_dataset)
#
#print(df['Id'][500])  #King Kong

#print(df['Genre'][500])  #Tagged as multiple Genres.

#Id and Genre are not labels to be trained. So drop them from the dataframe.
#No need to convert to categorical as the dataset is already in the right format.
#y = np.array(df.drop(['Id', 'Genre'], axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.3)

In [29]:
X_train.shape

(70, 640, 640, 3)

In [30]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 636, 636, 16)      1216      
                                                                 
 batch_normalization_8 (Batc  (None, 636, 636, 16)     64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 318, 318, 16)     0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 318, 318, 16)      0         
                                                                 
 conv2d_9 (Conv2D)           (None, 314, 314, 32)      12832     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 157, 157, 32)    

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:





#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


#################################################
#Validate on an image
#img = image.load_img('movie_dataset_multilabel/images/tt4425064.jpg', target_size=(SIZE,SIZE,3))
img = image.load_img('ddlj.jpg', target_size=(SIZE,SIZE,3))

img = image.img_to_array(img)
img = img/255.
plt.imshow(img)
img = np.expand_dims(img, axis=0)

classes = np.array(df.columns[2:]) #Get array of all classes
proba = model.predict(img)  #Get probabilities for each class
sorted_categories = np.argsort(proba[0])[:-11:-1]  #Get class names for top 10 categories

#Print classes and corresponding probabilities
for i in range(10):
    print("{}".format(classes[sorted_categories[i]])+" ({:.3})".format(proba[0][sorted_categories[i]]))

###################################################

_, acc = model.evaluate(X_test, y_test)
print("Accuracy = ", (acc * 100.0), "%")

################################################################






